# Decison Tree from Scratch!
We will be using Kaggle's [Spotify Song Attributes](https://www.kaggle.com/geomack/spotifyclassification/home) dataset. The dataset contains a number of features of songs from 2017 and a binary target variable representing whether the user liked the song or not. See the documentation of all the features from [here](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/).

In [1]:
import numpy as np
import pandas as pd

In [2]:
def entropy(p):
    """
    A helper function that computes the entropy of the 
    discrete distribution p (stored in a 1D numpy array).
    The elements of p should add up to 1.
    This function ensures lim p-->0 of p log(p) = 0
    which is mathematically true, but numerically results in NaN
    because log(0) returns -Inf.
    """
    plogp = 0*p # initialize full of zeros
    plogp[p>0] = p[p>0]*np.log(p[p>0]) # only do the computation when p>0
    return -np.sum(plogp)

class DecisionStump():

    def fit(self, X, y, split_features=None):        
        
        n, d = X.shape

        # Address the trivial case where we do not split
        count = np.bincount(y)

        # Compute total entropy (needed for information gain)
        p = count/np.sum(count); # Convert counts to probabilities
        entropyTotal = entropy(p)

        maxGain = 0
        self.splitVariable = None
        self.splitValue = None
        self.splitSat = np.argmax(count)
        self.splitNot = None

        # Check if labels are not all equal
        if np.unique(y).size <= 1:
            return

        if split_features is None:
            split_features = range(d)

        for j in split_features:
            thresholds = np.unique(X[:,j])
            for value in thresholds[:-1]:
                # Count number of class labels where the feature is greater than threshold
                y_vals = y[X[:,j] > value]
                countSat = np.bincount(y_vals, minlength=len(count))
                countNot = count-countSat
                                
                # Compute infogain
                pSat = countSat/np.sum(countSat)
                pNot = countNot/np.sum(countNot)
                HSat = entropy(pSat)
                HNot = entropy(pNot)
                probSat = np.sum(X[:,j] > value)/n
                probNot = 1-probSat
                
                infoGain = entropyTotal - probSat*HSat - probNot*HNot
                
                # Compare to minimum error so far
                if infoGain > maxGain:
                    # This is the highest information gain, store this value
                    maxGain = infoGain
                    splitVariable = j
                    splitValue = value
                    splitSat = np.argmax(countSat)
                    splitNot = np.argmax(countNot)
    
        self.splitVariable = splitVariable
        self.splitValue = splitValue
        self.splitSat = splitSat
        self.splitNot = splitNot

    def predict(self, X):
        splitVariable = self.splitVariable
        splitValue = self.splitValue
        splitSat = self.splitSat
        splitNot = self.splitNot

        m, d = X.shape

        if splitVariable is None:
            return splitSat * np.ones(m)

        yhat = np.zeros(m)

        for i in range(m):
            if X[i, splitVariable] > splitValue:
                yhat[i] = splitSat
            else:
                yhat[i] = splitNot
        return yhat

In [3]:
class DecisionTree():
    """
    Decision tree implementation in python using a
    basic implementation of a decision stump
    """

    def __init__(self, max_depth):

        self.max_depth = max_depth
        self.ds = DecisionStump()

    def fit(self, X, y):
        """
        Fits a decision tree using greedy recursive splitting
        """
        
        # Learn a decision stump
        self.ds.fit(X, y)

        # check if we reached max depth or the decision stump splits
        # no more, return the stum in these cases
        if self.max_depth <= 1 or not self.ds.splitVariable:
            self.leftModel = None
            self.rightModel = None
            return

        j = self.ds.splitVariable
        value = self.ds.splitValue

        # Find indices of examples in each split
        split_index_left = X[:, j] <= value
        split_index_right = X[:, j] > value

        # Fit a decision tree to each split, decreasing maximum depth by 1
        self.leftModel = DecisionTree(self.max_depth - 1)
        self.leftModel.fit(X[split_index_left], y[split_index_left])

        self.rightModel = DecisionTree(self.max_depth - 1)
        self.rightModel.fit(X[split_index_right], y[split_index_right])


    def predict(self, X):

        n, d = X.shape
        y = np.zeros(n)

        # If no further splitting, return the majority label
        if self.ds.splitVariable is None:
            return self.ds.splitSat * np.ones(n)

        # the case with depth=1, just a single stump
        elif self.leftModel == None:
            return self.ds.predict(X)

        else:
            # recurse on both the subtrees
            j = self.ds.splitVariable
            value = self.ds.splitValue

            split_index_left = X[:, j] <= value
            split_index_right = X[:, j] > value

            y[split_index_left] = self.leftModel.predict(X[split_index_left])
            y[split_index_right] = self.rightModel.predict(X[split_index_right])

        return y


In [4]:
spotify = pd.read_csv('/Users/adityasharma/Desktop/spotifyclassification.zip', 
                      compression='zip')
features = [x for x in spotify.columns.values if x not in ['target', 'Unnamed: 0', 'song_title', 'artist']]
print(features)

['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']


In [5]:
# separating the training data and lables
X = spotify.loc[:, features]
y = spotify.target

model_scratch_dt = DecisionTree(max_depth = 9999)
model_scratch_dt.fit(X.values, y.values)

pred = model_scratch_dt.predict(X.values)
print(np.sum(pred == y) / y.size)

0.880019831432821


In [6]:
# using the decision tree classifier from the sklearn library to fit the model
from sklearn import tree 
model = tree.DecisionTreeClassifier()
model.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [7]:
predictions = model.predict(X)
model.score(X, y)

0.9985126425384234

In [11]:
print("Training accuracy for sklearn decision tree is: {0}".format(model.score(X, y)))
print("Training accuracy for manual decision tree is: {0}".format(np.sum(pred == y)/y.size))

Training accuracy for sklearn decision tree is: 0.9985126425384234
Training accuracy for manual decision tree is: 0.880019831432821


In [9]:
# timing the sklearn decision tree
%timeit model.fit(X, y)

18.9 ms ± 187 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
# timing the custom made recursive decision tree
%timeit model_scratch_dt.fit(X.values, y.values)

6.3 s ± 66.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
